In [21]:
import pandas as pd
import numpy as np
import time

In [1]:
from sklearn import datasets

In [8]:
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
y = iris.target

In [11]:
from sklearn.metrics import roc_auc_score
import category_encoders as ce
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    train_size=0.8,
                                                    random_state=41)

In [72]:
from hyperopt import hp, tpe,space_eval
from hyperopt.fmin import fmin
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [17]:
cl1=DecisionTreeClassifier()

In [19]:
cl1.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [31]:
score_lgb=[]
params_lgb=[]

In [87]:
def hyperopt_lgb_score(params):
    clf = DecisionTreeClassifier(**params)
    # усреднение по 3ем фолдам, для уменьшения влияния стахостичности
    # для ускорения можно использовать train_test_split один раз
    current_score = cross_val_score(clf, X_train, y_train, scoring='neg_log_loss',cv=5).mean()
    print(current_score, params)
    score_lgb.append(current_score)
    params_lgb.append(params)
    return 1-current_score

In [88]:
space_lgb = {
            'ccp_alpha':hp.choice('ccp_alpha', np.arange(0, 12, 2, dtype=int)),
            'class_weight':hp.choice('class_weight', [None,'balanced']),
            'criterion':hp.choice('criterion', ['gini', 'entropy']),
            'max_depth':  hp.choice('max_depth', np.arange(1, 10, dtype=int)),
            'max_features':hp.choice('max_features', ['auto', 'sqrt','log2']),
            'max_leaf_nodes':hp.choice('max_leaf_nodes', np.arange(0, 1001, 10, dtype=int)),
            'min_samples_leaf':hp.choice('min_samples_leaf', np.arange(1, 101, 1, dtype=int)),
            'min_samples_split':hp.choice('min_samples_split', np.arange(1, 101, 1, dtype=int))
            #'num_leaves': hp.choice('num_leaves', np.arange(2, 64, 2, dtype=int)),
            #'metric':'auc',
            #'seed':1234,
            #'n_estimators': 100,#hp.choice('n_estimators', range(100, 3000)),
            #'learning_rate':hp.quniform('learning_rate', 0.001, 0.1, 0.001),
            #'max_depth':  hp.choice('max_depth', np.arange(1, 3, dtype=int)),
            #'min_child_weight': hp.choice('min_child_weight', np.arange(1, 300, dtype=int)),
            #'subsample': hp.uniform ('subsample', 0.5, 1),
            #'gamma': hp.uniform ('gamma', 0, 5),
            #'colsample_bytree': hp.uniform ('colsample_bytree', 0.5, 1),
            #'reg_alpha': hp.uniform ('reg_alpha', 1e-05,1),
            #'n_jobs': 8
}

In [89]:
%%time
x1=time.time()
best = fmin(fn=hyperopt_lgb_score, space=space_lgb, algo=tpe.suggest, max_evals=100)
x2=time.time()

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].



  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [75]:
best_model_param=space_eval(space_lgb,best)

In [86]:
best_model_param

{'ccp_alpha': 0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 7,
 'max_features': 'log2',
 'max_leaf_nodes': 890,
 'min_samples_leaf': 32,
 'min_samples_split': 34}

In [76]:
clf1=DecisionTreeClassifier(**best_model_param)

In [77]:
clf1.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0, max_depth=7, max_features='log2',
                       max_leaf_nodes=890, min_samples_leaf=32,
                       min_samples_split=34)

In [81]:
from sklearn.metrics import classification_report

In [85]:
print(classification_report(y_test, clf1.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.83      0.91      0.87        11
           2       0.89      0.80      0.84        10

    accuracy                           0.90        30
   macro avg       0.91      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30

